In [2]:
import os
from dotenv import load_dotenv
import openai
import pickle
import pandas as pd
import time

In [3]:
load_dotenv()

OPEN_AI_KEY = os.getenv('OPEN_AI_KEY')

In [4]:
openai.api_key = OPEN_AI_KEY


In [5]:
# report_pdf_content_dict = pd.read_pickle("../data/report_pdf_content_dict.pkl")

In [6]:
# len(report_pdf_content_dict)

In [7]:
github_df = pd.read_csv("../data/github_df.csv", index_col=0)
github_df["readme"].isnull().sum()

4

In [8]:
non_null_github_dfs = github_df[~github_df["readme"].isnull()]

In [9]:
def split_text(text):
    max_chunk_size = 2048
    chunks = []
    current_chunk = ""
    for sentence in text.split("."):
        if len(current_chunk) + len(sentence) < max_chunk_size:
            current_chunk += sentence + "."
        else:
            chunks.append(current_chunk.strip())
            current_chunk = sentence + "."
    if current_chunk:
        chunks.append(current_chunk.strip())
    return chunks

In [10]:
# Request the summarization using ChatGPT
# summarization_response = openai.Completion.create(
#     engine="text-davinci-002",
#     prompt="Please summarize the following text",
#     max_tokens=max_tokens,
#     api_key=OPEN_AI_KEY
# )

# https://medium.com/muthoni-wanyoike/implementing-text-summarization-using-openais-gpt-3-api-dcd6be4f6933
# Code from here: https://www.kdnuggets.com/2023/04/text-summarization-development-python-tutorial-gpt35.html
def generate_summarizer(text):
    
    res = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-16k",
        temperature=0.7,
        top_p=0.5,
        frequency_penalty=0.5,
        messages=
    [
        {
        "role": "system",
        "content": "You are a helpful assistant for text summarization.",
        },
        {
        "role": "user",
        "content": f"Summarize this: {text}",
        },
        ],
    )
    return (res["choices"][0]["message"]["content"])
        

In [11]:
non_null_github_dfs

,github_url,readme,contributors,language_breakdown
0,https://github.com/DSC-Capstone/projects-2019-...,# wiki-capstone\nPublic repository for DSC 180...,NaN,NaN
1,https://github.com/DSC-Capstone/projects-2019-...,# DSC180B Wikipedia Engagement\n\nThis project...,NaN,NaN
2,https://github.com/DSC-Capstone/projects-2019-...,# DSC180B Final Project: Investigating the Bia...,NaN,NaN
3,https://github.com/DSC-Capstone/projects-2019-...,# Name That Raga: Classification and Analysis ...,NaN,NaN
4,https://github.com/DSC-Capstone/projects-2019-...,"<p align=""center"">\n <!-- <b style=""font-si...",NaN,NaN
...,...,...,...,...
203,https://github.com/nahmann/DSC180-B16,# DSC 180 B16 - Decentralized Location Consens...,"{'nahmann': 'https://github.com/nahmann', 'Ale...","{'HTML': 0.49, 'Jupyter Notebook': 0.31, 'Pyth..."
204,https://github.com/acanonig/DSC180B-Proxensus-,# DSC 180B Project Code - Proxensus\nGroup Mem...,{'tjuacalla123': 'https://github.com/tjuacalla...,{'Java': 1.0}
205,https://github.com/pnagasam/dsc180a_capstone_p...,# dsc180a_capstone_project\n\nThis program tak...,{'Lalaluke413': 'https://github.com/Lalaluke41...,"{'Jupyter Notebook': 1.0, 'Python': 0.0}"
206,https://github.com/BillChen24/DSC180B-Project-...,# DSC180B-Project-B319-2\n# Domain Adaptation ...,"{'Alina-Zhi': 'https://github.com/Alina-Zhi', ...","{'Jupyter Notebook': 0.99, 'Python': 0.01, 'Do..."


In [12]:
# len(non_null_github_dfs["readme"][0])

In [17]:
# report_text_dict = {}
# report_text_dict = pd.read_pickle("../data/report_text_dict.pkl")
# not_finished_text = []
# i = 0
# for k, v in report_pdf_content_dict.items():
#     if i < 46:
#         i += 1
#     else:
#         text = v[0]
#         try:
#             summary_text = generate_summarizer(text)
#             report_text_dict[k] = [text, summary_text]
#         except openai.InvalidRequestError:
#             print("Invalid Request Error")
#             not_finished_text.append(k)

#         print(f"{k} Done. Waiting 10 seconds")
#         time.sleep(10)
#         i += 1

report_readme_dict = pd.read_pickle("../data/report_readme_dict.pkl")
not_finished_readmes = []
for i, row in non_null_github_dfs.iterrows():
    if row["github_url"] not in report_readme_dict:
        text = row["readme"]
        try:
            summary_text = generate_summarizer(text)
            report_readme_dict[row["github_url"]] = [text, summary_text]
            print(f"{row['github_url']} Done. Waiting 15 seconds")
            # time.sleep(15)
        except openai.InvalidRequestError:
            print("Invalid Request Error")
            not_finished_readmes.append(row["github_url"])
        

https://github.com/nickthegroot/recipe-recommendation Done. Waiting 15 seconds
https://github.com/mjw49/DSC180B-Quarter-2-Project Done. Waiting 15 seconds
https://github.com/camille-004/Graph-HSCN Done. Waiting 15 seconds
https://github.com/bliu8923/dsc180b-project Done. Waiting 15 seconds


In [21]:
report_readme_dict

{'https://github.com/DSC-Capstone/projects-2019-2020/tree/master/project_01/': ['# wiki-capstone\nPublic repository for DSC 180B senior capstone project exploring racial bias in Oscars and Golden Globes Award Shows.\n\nHow to Run:\n  This project consists of two parts: data collection and creating visuals. Both "test-project" and "full-project" targets\n  will run both parts. \n\n  Targets:\n  1. "clean" : Removes any test data created and removes any visualizations created. Does not remove full data because that is needed for the visualizations.\n  2. "test-project": Runs the data collection process for only years 1934-1935. It also creates visuals using data from years 1934-2008.\n  3. "full-project": Runs the data collection process for all years 1934- 2008. It also creates visuals using this data. (Delete "data/" folder before running this target)\n  \n  \n',
  'This is a public repository for a senior capstone project called "wiki-capstone" that focuses on exploring racial bias in

In [24]:
# list(report_pdf_content_dict.items())[2][1]
report_readme_dict

# open a file, where you ant to store the data
file = open("../data/report_readme_dict.pkl", 'wb')

# dump information to that file
pickle.dump(report_readme_dict, file)

# close the file
file.close()

In [26]:
(pd.read_pickle("../data/report_readme_dict.pkl"))

{'https://github.com/DSC-Capstone/projects-2019-2020/tree/master/project_01/': ['# wiki-capstone\nPublic repository for DSC 180B senior capstone project exploring racial bias in Oscars and Golden Globes Award Shows.\n\nHow to Run:\n  This project consists of two parts: data collection and creating visuals. Both "test-project" and "full-project" targets\n  will run both parts. \n\n  Targets:\n  1. "clean" : Removes any test data created and removes any visualizations created. Does not remove full data because that is needed for the visualizations.\n  2. "test-project": Runs the data collection process for only years 1934-1935. It also creates visuals using data from years 1934-2008.\n  3. "full-project": Runs the data collection process for all years 1934- 2008. It also creates visuals using this data. (Delete "data/" folder before running this target)\n  \n  \n',
  'This is a public repository for a senior capstone project called "wiki-capstone" that focuses on exploring racial bias in